# AIX360框架LIME使用方法

PS: 实际上AIX360直接调用的LIME工具包的内容，并没有作太多的修改，因此可以直接调用LIME工具包来使用LIME解释模型。

**安装方法:**</br>
``` python
conda create --name aix360 python=3.6
conda activate aix360
(aix360)$ git clone https://github.com/Trusted-AI/AIX360
(aix360)$ pip install -e .
```

**anaconda开始没有切换环境的功能** </br>
```
$conda install nb_conda
```

In [10]:
from __future__ import print_function
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost
# Importing LimeTextExplainer (aix360 sytle)
from aix360.algorithms.lime import LimeTabularExplainer
from aix360.algorithms.lime import LimeTextExplainer
# Supress jupyter warnings if required for cleaner output
import warnings
warnings.simplefilter('ignore')

## 读取自生成数据

In [2]:
df = pd.read_csv("../train/train_data_0.csv")  #10000x100数据，不包含Target
df

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature90,feature91,feature92,feature93,feature94,feature95,feature96,feature97,feature98,feature99
0,84.071517,15,95,37,109.222187,41.054423,14,12.635977,14.010126,70,...,26,56,32,30,30,65,36.400093,55,44,11.299354
1,64.376450,12,39,40,108.267721,58.627029,46,80.431019,3.598875,70,...,28,38,69,28,10,44,32.313545,30,45,10.109159
2,61.747596,36,58,7,51.315514,10.865075,53,43.284407,52.822478,70,...,20,45,62,43,29,48,35.294801,27,70,11.926330
3,108.096804,32,73,59,40.416383,12.580249,70,19.908695,59.408556,70,...,26,7,57,68,3,42,37.391035,47,71,10.819648
4,93.659688,14,45,30,85.312636,47.390543,81,30.780931,44.901262,71,...,30,10,46,42,3,42,32.743209,42,47,11.301096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,46.443361,29,59,22,41.987071,57.629751,39,78.792761,4.834577,71,...,26,63,38,66,16,62,36.587602,31,50,7.942675
9996,101.995631,23,57,16,100.037233,41.170912,25,32.059358,31.253103,71,...,29,57,44,52,7,62,37.806850,31,70,11.128170
9997,79.100888,42,36,44,100.913728,59.806790,17,69.167360,47.454068,70,...,22,60,46,4,32,62,32.413842,52,59,10.260244
9998,97.844526,31,41,46,52.108113,51.004928,65,39.811859,36.990671,70,...,25,20,42,4,19,47,36.201993,27,54,8.643205


In [3]:
def normalize(x):
    x_max = x.max()
    x_min = x.min()
    if x.max() == x.min():
        return x
    else:
        result = (x-x_min)/(x_max-x_min)
        return result

In [4]:
df["label"] = df.feature0 *1 + df.feature1 * 1 + df.feature3*1
df["label"] = (normalize(df["label"])>0.5).astype(int)
df

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature91,feature92,feature93,feature94,feature95,feature96,feature97,feature98,feature99,label
0,84.071517,15,95,37,109.222187,41.054423,14,12.635977,14.010126,70,...,56,32,30,30,65,36.400093,55,44,11.299354,0
1,64.376450,12,39,40,108.267721,58.627029,46,80.431019,3.598875,70,...,38,69,28,10,44,32.313545,30,45,10.109159,0
2,61.747596,36,58,7,51.315514,10.865075,53,43.284407,52.822478,70,...,45,62,43,29,48,35.294801,27,70,11.926330,0
3,108.096804,32,73,59,40.416383,12.580249,70,19.908695,59.408556,70,...,7,57,68,3,42,37.391035,47,71,10.819648,1
4,93.659688,14,45,30,85.312636,47.390543,81,30.780931,44.901262,71,...,10,46,42,3,42,32.743209,42,47,11.301096,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,46.443361,29,59,22,41.987071,57.629751,39,78.792761,4.834577,71,...,63,38,66,16,62,36.587602,31,50,7.942675,0
9996,101.995631,23,57,16,100.037233,41.170912,25,32.059358,31.253103,71,...,57,44,52,7,62,37.806850,31,70,11.128170,1
9997,79.100888,42,36,44,100.913728,59.806790,17,69.167360,47.454068,70,...,60,46,4,32,62,32.413842,52,59,10.260244,1
9998,97.844526,31,41,46,52.108113,51.004928,65,39.811859,36.990671,70,...,20,42,4,19,47,36.201993,27,54,8.643205,1


In [5]:
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,:-1],df.iloc[:,-1],test_size = 0.3, random_state = 42)

In [6]:
data = df.iloc[:,:-1]
label = df.iloc[:,-1]
X_train,X_test, y_train,y_test = train_test_split(data,label,test_size = 0.3)

## 使用AIX360封装函数调用LIME

**使用多项式朴素贝叶斯MultinomialNB**(用不了，只能处理整数特征）

In [7]:
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.preprocessing import MinMaxScaler
xgb_clf = xgboost.XGBClassifier()
xgb_clf.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [8]:
limeexplainer = LimeTabularExplainer(X_train.values,feature_names=data.columns,discretize_continuous=True,discretizer="quartile",verbose=True,mode = "classification")
exp = limeexplainer.explain_instance(X_test.iloc[0], xgb_clf.predict_proba, num_features=10)
print('Predicted class = %d' % xgb_clf.predict(test_data))
print('True class: %s' % true_data.values )

ValueError: feature_names mismatch: ['feature0', 'feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6', 'feature7', 'feature8', 'feature9', 'feature10', 'feature11', 'feature12', 'feature13', 'feature14', 'feature15', 'feature16', 'feature17', 'feature18', 'feature19', 'feature20', 'feature21', 'feature22', 'feature23', 'feature24', 'feature25', 'feature26', 'feature27', 'feature28', 'feature29', 'feature30', 'feature31', 'feature32', 'feature33', 'feature34', 'feature35', 'feature36', 'feature37', 'feature38', 'feature39', 'feature40', 'feature41', 'feature42', 'feature43', 'feature44', 'feature45', 'feature46', 'feature47', 'feature48', 'feature49', 'feature50', 'feature51', 'feature52', 'feature53', 'feature54', 'feature55', 'feature56', 'feature57', 'feature58', 'feature59', 'feature60', 'feature61', 'feature62', 'feature63', 'feature64', 'feature65', 'feature66', 'feature67', 'feature68', 'feature69', 'feature70', 'feature71', 'feature72', 'feature73', 'feature74'

In [14]:
explainer = LimeTabularExplainer(X_train.values,feature_names=data.columns,discretize_continuous=True,discretizer="quartile",verbose=True,mode = "classification")
print(type(explainer))

<class 'aix360.algorithms.lime.lime_wrapper.LimeTabularExplainer'>


**报错原因为0.1.37版本LIME的bug，单纯使用LIME 0.2以上版本不会出现这个问题**